In [ ]:
%%HTML
<style>
    body {
        --vscode-font-family: "Noto Serif"
    }
</style>

# Segment Tree vs Fenwick Tree (Binary Indexed Tree)

Both are **data structures** designed for efficient **range queries and updates** on arrays.

---

## 1. Segment Tree

- A **binary tree** structure that stores information about intervals (segments) of an array.  
- Each node represents a segment `[L, R]` and stores an aggregate (e.g., sum, min, max).  
- Root covers the entire array, children cover halves.  

**Operations:**
- Query (e.g., sum of [l, r]): O(log n)  
- Update (point update or range update): O(log n)  
- Memory usage: O(4n)  

**Use cases:**
- Range queries (sum, min, max, gcd).  
- Range updates with **lazy propagation**.  
- Problems like **RMQ (Range Minimum Query)**, **interval sum**, **competitive programming queries**.  

---

## 2. Fenwick Tree (Binary Indexed Tree, BIT)

- A **compact array-based structure** that supports prefix queries using binary decomposition.  
- Each index covers a segment of length equal to the **lowest set bit** of that index.  

**Operations:**
- Prefix sum query: O(log n)  
- Range query [l, r]: prefix(r) - prefix(l-1)  
- Update: O(log n)  
- Memory usage: O(n)  

**Use cases:**
- Fast **prefix/range sum queries**.  
- Situations with frequent updates + queries.  
- Simpler and more space-efficient than segment trees.  

---

## 3. Comparison

| Feature            | Segment Tree | Fenwick Tree (BIT) |
|--------------------|--------------|---------------------|
| Query time         | O(log n)     | O(log n)            |
| Update time        | O(log n)     | O(log n)            |
| Memory             | O(4n)        | O(n)                |
| Complexity         | More complex | Simple              |
| Supports           | Sum, min, max, gcd, range updates (lazy) | Mostly prefix sums (extensions possible) |
| Use cases          | Advanced range queries, competitive problems | Prefix sums, frequency tables, simpler problems |

---

✅ **Rule of thumb:**
- If you only need **range sum queries and point updates** → **Fenwick Tree (BIT)** (simpler, faster in practice).  
- If you need **min/max, gcd, or range updates** → **Segment Tree** (more powerful).  

In [ ]:
package main

import (
    "fmt"
    "math"
)

// SegmentTree represents a generic segment tree
type SegmentTree[T any] struct {
    n    int
    size int
    tree []T
    op   func(T, T) T // associative binary operation
    id   T            // identity element
}

// NewSegmentTree creates a new segment tree
func NewSegmentTree[T any](data []T, op func(T, T) T, identity T) *SegmentTree[T] {
    n := len(data)
    if n == 0 {
        return &SegmentTree[T]{
            n:    0,
            size: 1,
            tree: []T{identity},
            op:   op,
            id:   identity,
        }
    }
    
    // Find next power of 2
    size := 1
    for size < n {
        size <<= 1
    }
    
    st := &SegmentTree[T]{
        n:    n,
        size: size,
        tree: make([]T, 2*size),
        op:   op,
        id:   identity,
    }
    
    // Initialize leaves
    for i := 0; i < n; i++ {
        st.tree[size+i] = data[i]
    }
    // Fill padding with identity
    for i := size + n; i < 2*size; i++ {
        st.tree[i] = identity
    }
    // Build internal nodes
    for i := size - 1; i > 0; i-- {
        st.tree[i] = op(st.tree[2*i], st.tree[2*i+1])
    }
    
    return st
}

// Len returns the size of the underlying array
func (st *SegmentTree[T]) Len() int {
    return st.n
}

// Get returns the value at index i
func (st *SegmentTree[T]) Get(i int) T {
    if i < 0 || i >= st.n {
        panic("index out of bounds")
    }
    return st.tree[st.size+i]
}

// Set updates the value at index i
func (st *SegmentTree[T]) Set(i int, value T) {
    if i < 0 || i >= st.n {
        panic("index out of bounds")
    }
    
    idx := st.size + i
    st.tree[idx] = value
    idx /= 2
    
    for idx >= 1 {
        st.tree[idx] = st.op(st.tree[2*idx], st.tree[2*idx+1])
        idx /= 2
    }
}

// Query computes the operation over range [l, r] (inclusive)
func (st *SegmentTree[T]) Query(l, r int) T {
    if st.n == 0 {
        panic("cannot query empty segment tree")
    }
    if l > r {
        return st.id
    }
    if l < 0 || r >= st.n {
        panic("range out of bounds")
    }
    
    resLeft := st.id
    resRight := st.id
    l += st.size
    r += st.size
    
    for l <= r {
        if l%2 == 1 {
            resLeft = st.op(resLeft, st.tree[l])
            l++
        }
        if r%2 == 0 {
            resRight = st.op(st.tree[r], resRight)
            r--
        }
        l /= 2
        r /= 2
    }
    
    return st.op(resLeft, resRight)
}

// FenwickTree represents a Binary Indexed Tree
type FenwickTree[T any] struct {
    n   int
    bit []T
    op  func(T, T) T // associative and commutative operation
    inv func(T) T    // inverse function
    id  T            // identity element
}

// NewFenwickTree creates a new Fenwick tree
func NewFenwickTree[T any](data []T, op func(T, T) T, inv func(T) T, identity T) *FenwickTree[T] {
    n := len(data)
    ft := &FenwickTree[T]{
        n:   n,
        bit: make([]T, n+1),
        op:  op,
        inv: inv,
        id:  identity,
    }
    
    // Initialize with identity
    for i := 0; i <= n; i++ {
        ft.bit[i] = identity
    }
    
    // Add all initial values
    for i, v := range data {
        ft.Add(i, v)
    }
    
    return ft
}

// Len returns the size
func (ft *FenwickTree[T]) Len() int {
    return ft.n
}

// Add performs point add: arr[i] = op(arr[i], delta)
func (ft *FenwickTree[T]) Add(i int, delta T) {
    if i < 0 || i >= ft.n {
        panic("index out of bounds")
    }
    
    idx := i + 1
    for idx <= ft.n {
        ft.bit[idx] = ft.op(ft.bit[idx], delta)
        idx += idx & -idx
    }
}

// Prefix computes the operation over [0, r] inclusive
func (ft *FenwickTree[T]) Prefix(r int) T {
    if r < 0 {
        return ft.id
    }
    if r >= ft.n {
        r = ft.n - 1
    }
    
    res := ft.id
    idx := r + 1
    for idx > 0 {
        res = ft.op(res, ft.bit[idx])
        idx -= idx & -idx
    }
    return res
}

// RangeQuery computes the operation over [l, r] via prefix sums
func (ft *FenwickTree[T]) RangeQuery(l, r int) T {
    if l > r {
        return ft.id
    }
    if l < 0 || r >= ft.n {
        panic("range out of bounds")
    }
    
    return ft.op(ft.Prefix(r), ft.inv(ft.Prefix(l-1)))
}

// Get returns the current value at index i
func (ft *FenwickTree[T]) Get(i int) T {
    return ft.RangeQuery(i, i)
}

// Set sets arr[i] to value by computing required delta
func (ft *FenwickTree[T]) Set(i int, value T) {
    current := ft.Get(i)
    // delta such that op(current, delta) == value
    // => delta = op(inv(current), value)
    delta := ft.op(ft.inv(current), value)
    ft.Add(i, delta)
}

// Helper functions
func intAdd(a, b int) int { return a + b }
func intNeg(a int) int    { return -a }
func intMin(a, b int) int {
    if a < b {
        return a
    }
    return b
}
func intMax(a, b int) int {
    if a > b {
        return a
    }
    return b
}

## Examples

Below are examples showing segment tree and Fenwick tree usage.

In [ ]:
%%
// Segment Tree examples
arr := []int{2, 1, 3, 4, 5, 2, 7}

// Range Sum Segment Tree
sumSeg := NewSegmentTree(arr, intAdd, 0)
fmt.Println("sum [0, 6] =", sumSeg.Query(0, 6)) // 24
fmt.Println("sum [2, 4] =", sumSeg.Query(2, 4)) // 12
sumSeg.Set(3, 10) // arr[3] = 10
fmt.Println("after update arr[3]=10, sum [2, 4] =", sumSeg.Query(2, 4)) // 18

// Range Min Segment Tree
arr2 := []int{5, 3, 6, 2, 8, 7}
minSeg := NewSegmentTree(arr2, intMin, math.MaxInt)
fmt.Println("min [0, 5] =", minSeg.Query(0, 5)) // 2
fmt.Println("min [1, 3] =", minSeg.Query(1, 3)) // 2
minSeg.Set(3, 9)
fmt.Println("after update arr2[3]=9, min [1, 3] =", minSeg.Query(1, 3)) // 3

// Range Max Segment Tree
arr3 := []int{5, 3, 6, 2, 8, 7}
maxSeg := NewSegmentTree(arr3, intMax, math.MinInt)
fmt.Println("max [0, 5] =", maxSeg.Query(0, 5)) // 8
fmt.Println("max [2, 4] =", maxSeg.Query(2, 4)) // 8
maxSeg.Set(4, 1)
fmt.Println("after update arr3[4]=1, max [2, 4] =", maxSeg.Query(2, 4)) // 6

In [ ]:
%%
// Fenwick Tree examples
arr := []int{2, 1, 3, 4, 5, 2, 7}
bit := NewFenwickTree(arr, intAdd, intNeg, 0)

fmt.Println("prefix(6) =", bit.Prefix(6))       // 24
fmt.Println("range_query(2, 4) =", bit.RangeQuery(2, 4))  // 12
bit.Add(3, 6)  // arr[3] += 6 -> 10
fmt.Println("after add(3,6), range_query(2, 4) =", bit.RangeQuery(2, 4))  // 18
bit.Set(0, 5)  // arr[0] = 5
fmt.Println("after set(0,5), prefix(2) =", bit.Prefix(2))  // 5 + 1 + 3 = 9